<a href="https://colab.research.google.com/github/aviax1/AE2/blob/master/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**set data and model**

In [ ]:
!pip install wandb

**set models**

In [73]:
import torch,wandb,os,warnings,csv
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
from tensorflow.keras.datasets import mnist
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

num_epochs=6           #
batch_size = 64        #
image_size=784         #
hidden_size=124        #
lv_size = 64           # Latent Variable 
learning_rate=1e-4     #
cret = nn.MSELoss()    # criterion
warnings.filterwarnings('ignore')

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(image_size, hidden_size), 
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, lv_size))
        self.decoder = nn.Sequential(
            nn.Linear(lv_size, hidden_size),nn.ReLU(True),
            nn.Linear(hidden_size, hidden_size),nn.ReLU(True),
            nn.Linear(hidden_size, hidden_size),nn.ReLU(True),
            nn.Linear(hidden_size, hidden_size),nn.ReLU(True),
            nn.Linear(hidden_size, image_size), nn.Tanh())
    def forward(self, x):
        return self.decoder(self.encoder(x))

def model_name(digit):
  return './ae_'+str(digit)+'.pth'


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-6)
class DigitDataSet(Dataset):
  def __init__(self, dataset):
      self.dataset = dataset
      self.transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5], [0.5])])
  def __len__(self):
      return len(self.dataset)
  def __getitem__(self, idx):
      if torch.is_tensor(idx):
          idx = idx.tolist()
      return self.transform( self.dataset[idx,:,:])

def save_model(digit,model):
  mn=model_name(digit)
  torch.save(model.state_dict(),mn )
  wandb.save(mn)
  print("save model "+ mn)

def load_model_ifexist(digit,model):
  mn=model_name(digit)
  if os.path.isfile(mn):
    model.load_state_dict(torch.load(mn))
    model.eval()
  return model

def train_by_digit(by_digit,dna,model,ne=num_epochs,opt=optimizer):
  db=xtrain[dna==by_digit]
  db=db[0:1000,:,:]
  dataloader = DataLoader(DigitDataSet(db), batch_size=batch_size,shuffle=True, num_workers=6)
  for epoch in range(ne):
    for data in dataloader:
      imgs = Variable(data.view(data.size(0), -1))
      output_imgs = model(imgs)
      loss = cret(output_imgs, imgs)
      opt.zero_grad()
      loss.backward()
      opt.step()
    print('epoch [{}/{}], loss:{:.4f}' .format(epoch + 1, ne, loss.data))
  return loss.data

def dns_score(dna,model):
  scoresum=0
  for i in range(10):
    scorei=train_by_digit(i,dna,model)
    scoresum+=scorei
    #print("test "+str(i)+"score="+str(scorei))
  scoresum=scoresum/10
  print("scoresum"+str(scoresum))
  return scoresum

**set dna**

In [75]:


def dna_init(n):
  arr = np.zeros( n,dtype=np.int32)
  for i in range(n):
    arr[i]=random.randint(0,9)
  return arr

def mutation(dna):
  newdna=dna

(xtrain,ytrain), (xtest,ytest) = mnist.load_data()
dna = dna_init(len(xtrain))
for i in range(10):
  model = autoencoder()
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model.to(device)
  
  score =dns_score(dna,model)
  #childrens = number of mutation ,chose the best

epoch [1/6], loss:0.9302
epoch [2/6], loss:0.9309
epoch [3/6], loss:0.9311
epoch [4/6], loss:0.9342
epoch [5/6], loss:0.9328
epoch [6/6], loss:0.9310
epoch [1/6], loss:0.9301
epoch [2/6], loss:0.9320
epoch [3/6], loss:0.9239
epoch [4/6], loss:0.9322
epoch [5/6], loss:0.9321
epoch [6/6], loss:0.9331
epoch [1/6], loss:0.9284
epoch [2/6], loss:0.9304
epoch [3/6], loss:0.9287
epoch [4/6], loss:0.9275
epoch [5/6], loss:0.9298
epoch [6/6], loss:0.9336
epoch [1/6], loss:0.9284
epoch [2/6], loss:0.9307
epoch [3/6], loss:0.9322
epoch [4/6], loss:0.9290
epoch [5/6], loss:0.9297
epoch [6/6], loss:0.9277
epoch [1/6], loss:0.9300
epoch [2/6], loss:0.9294
epoch [3/6], loss:0.9304
epoch [4/6], loss:0.9298
epoch [5/6], loss:0.9261
epoch [6/6], loss:0.9241
epoch [1/6], loss:0.9264
epoch [2/6], loss:0.9279
epoch [3/6], loss:0.9303
epoch [4/6], loss:0.9347
epoch [5/6], loss:0.9282
epoch [6/6], loss:0.9320
epoch [1/6], loss:0.9317
epoch [2/6], loss:0.9308
epoch [3/6], loss:0.9268
epoch [4/6], loss:0.9263


KeyboardInterrupt: ignored

In [67]:
def testscore(dna,ytrain):
  return len(dna[dna==ytrain])/len(ytrain)

testscore(dna,ytrain)

0.10043333333333333